In [ ]:
from PIL import Image, ImageEnhance
import numpy as np
from PIL import ImageFilter
import colorsys
import os
from skimage.filters import gabor, gaussian
from IPython.display import display 
from matplotlib.pyplot import imshow
from pywt import dwt2
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import cv2

%matplotlib inline

In [ ]:
def enhance_brightness(image):
    
    mean_brightness = get_brightness(image)
    a, b = [0.3, 1]
    if mean_brightness<0.1:
        a = 0.1
    min_, max_ = [0, 1]
    new_brightness = (b - a) * (mean_brightness - min_) / (max_ - min_) + a
        
    brightness_factor = new_brightness/mean_brightness
    enhancer = ImageEnhance.Brightness(image)
    enhanced_image = enhancer.enhance(brightness_factor)
    return enhanced_image


def get_brightness(image):
   
    brightness = []
    pixel_values = list(image.getdata())
    for values in pixel_values:
        R, G, B = values
        bright = np.sqrt(0.299 * R ** 2 + 0.587 * G ** 2 + 0.114 * B ** 2) / 255
        brightness.append(bright)
    return np.mean(brightness)

In [ ]:
def get_image_energy(pixels):
    
    _, (cH, cV, cD) = dwt2(pixels.T, 'db1')
    energy = (cH ** 2 + cV ** 2 + cD ** 2).sum() / pixels.size
    return energy


def get_energy_density(pixels):
    
    energy = get_image_energy(pixels)
    energy_density = energy / (pixels.shape[0]*pixels.shape[1])
    return round(energy_density*100,5) # multiplying by 100 because the values are very small

In [ ]:
def get_magnitude(response):
    
    magnitude = np.array([np.sqrt(response[0][i][j]**2+response[1][i][j]**2)
                        for i in range(len(response[0])) for j in range(len(response[0][i]))])
    return magnitude

In [ ]:
def apply_pca(array):
    # apply dimensionality reduction to the input array
    standardized_data = StandardScaler().fit_transform(array)
    pca = PCA(n_components=1)
    pca.fit(standardized_data)
    transformed_data = pca.transform(standardized_data)
    return transformed_data

In [ ]:
def applygabor(image1):
    image = Image.open(image1).convert('RGB')
    image_size = image.size
    print(image_size)
    converter = ImageEnhance.Color(image)
    image = converter.enhance(0.5) #enchance color by a factor of 0.5
    image = enhance_brightness(image)
    # convert to grayscale
    image = image.convert('L')
    pixels = np.asarray(image, dtype="int32")
    energy_density = get_energy_density(pixels)
    # get fixed bandwidth using energy density
    bandwidth = abs(0.4*energy_density - 0.5)
    magnitude_dict = {}
    for theta in np.arange(0, np.pi, np.pi / 6):
        for freq in np.array([1.4142135623730951, 2.414213562373095, 2.8284271247461903, 3.414213562373095]): 
            filt_real, filt_imag = gabor(image, frequency=freq, bandwidth=bandwidth, theta=theta)
            # get magnitude response
            magnitude = get_magnitude([filt_real, filt_imag])
            ''' uncomment the lines below to visualize each magnitude response '''
            # im = Image.fromarray(magnitude.reshape(image_size)).convert('L')
            # display(im)
            magnitude_dict[(theta, freq)] = magnitude.reshape(image.size)
        # apply gaussian smoothing
    gabor_mag = []
    for key, values in magnitude_dict.items():
        # the value of sigma is chosen to be half of the applied frequency
        sigma = 0.5*key[1]
        smoothed = gaussian(values, sigma = sigma)
        gabor_mag.append(smoothed)
    gabor_mag = np.array(gabor_mag)

    # reshape so that we can apply PCA
    value = gabor_mag.reshape((-1, image_size[0]*image_size[1]))

    # get dimensionally reduced image
    pcaed = apply_pca(value.T).astype(np.uint8)
    result = pcaed.reshape((image_size[0], image_size[1]))
    result_im = Image.fromarray(result, mode='L')
    cv2.imwrite("/content/drive/MyDrive/Melanoma_Data/Gabor+Shift/train/NotMelanomaa/"+str(i)+".jpg", result)

In [ ]:
i=401
while(i<=700):
  applygabor("/content/drive/MyDrive/Melanoma_Data/Shift_eff/train/NotMelanoma/"+str(i)+".jpg")
  print(i)
  i=i+1

(224, 224)
401
(224, 224)
402
(224, 224)
403
(224, 224)
404
(224, 224)
405
(224, 224)
406
(224, 224)
407
(224, 224)
408
(224, 224)
409
(224, 224)
410
(224, 224)
411
(224, 224)
412
(224, 224)
413
(224, 224)
414
(224, 224)
415
(224, 224)
416
(224, 224)
417
(224, 224)
418
(224, 224)
419
(224, 224)
420
(224, 224)
421
(224, 224)
422
(224, 224)
423
(224, 224)
424
(224, 224)
425
(224, 224)
426
(224, 224)
427
(224, 224)
428
(224, 224)
429
(224, 224)
430
(224, 224)
431
(224, 224)
432
(224, 224)
433
(224, 224)
434
(224, 224)
435
(224, 224)
436
(224, 224)
437
(224, 224)
438
(224, 224)
439
(224, 224)
440
(224, 224)
441
(224, 224)
442
(224, 224)
443
(224, 224)
444
(224, 224)
445
(224, 224)
446
(224, 224)
447
(224, 224)
448
(224, 224)
449
(224, 224)
450
(224, 224)
451
(224, 224)
452
(224, 224)
453
(224, 224)
454
(224, 224)
455
(224, 224)
456
(224, 224)
457
(224, 224)
458
(224, 224)
459
(224, 224)
460
(224, 224)
461
(224, 224)
462
(224, 224)
463
(224, 224)
464
(224, 224)
465
(224, 224)
466
(224, 224)